# AMNH CG2 | Class 09 - Functional Annotation | Part 2: BLAST in BioPython
**Víctor Sojo** | `vsojo@amnh.org`<br/>

One way to find information about genes is using BLAST. You very likely already  know what BLAST is, but, very briefly, it looks for a _query_ sequence against a _target_ database, by performing extremely quick (and therefore not maximally accurate) alignments. This of course works best the more closely related the target sequences are to the query ones in the database. In an ideal scenario, if you have about the _subject_ (target) sequence and you're confident that the relationship between your query and this subject is solid, then you can infer that your own sequence probably has a similar physiological role to that of the sequence it looks like.

The danger is that, since sequences probably all share some sort of relationship to each other because of the single origin of life, then there will theoretically always be a point at which you can infer that two sequences are related. The question is: how related are they? And is this relevant for my study? i.e., is this BLAST _hit_ informative? There is no universal answer to this question, so you will need to proceed with care.

Here are some important BLAST terms, to the best of my understanding:
* **BLAST:** The **B**asic **L**ocal **A**lignment **S**earch **T**ool; it does what it says on the tin, i.e. it performs extremely basic (and therefore extremely fast, if not maximally accurate) local alignments to search for sequences of text against other sequences of text.
* **query:**  The sequence you want to find information for. Typically it comes from the organism you are studying.
* **target:** The database against which you are looking. A bigger database should provide a higher likelihood of finding something, but also a higher the likelihood of finding something spurious or not entirely relevant.
* **hit:** A sequence in the database that is inferred to be similar enough to the query sequence.
* **subject:** Related to the above, it's the sequence in the database that was hit.
* **score:** the score of the alignment between the query and the subject sequences.
* **e-value:** ...where to begin... this refers to the probability of finding a hit like the one you've found, or even better, at random, in the database you're using. It therefore depends on how long your query sequence is, how big the database, the composition of both your query and the database, how long the sequences in the database are, and so on. It is therefore **not a universal measure of quality**. Mind you, it _is_ a measure of quality, but it is _not_ universal, i.e. you cannot trivially compare one e-value for one search with another e-value for a different search.
* **program/tool:** which actual program you are using.
* **task:** Each BLAST program has an additional 

**References**: [This tutorial](http://dmnfarrell.github.io/bioinformatics/genome-annotation-python) was somewhat helpful in preparing this lesson, as were the general [BioPython reference](https://biopython.org/) and the [Biostar Handbook](https://biostar.myshopify.com/).

## Types of blast algorithms/programs/tools
There are actually several types of BLAST "tools", depending on what you're looking for (the _query_) and where you're looking for it (the _target_ database).

**Obvious note on molecular genetics:** because it is trivial to translate DNA/RNA to protein (_as long as you know the reading frame and strand!_), you can search for nucleotides in peptide space (i.e. amino-acid or protein space). The reverse is not true. Because the genetic code is _degenerate_, you cannot know which of the six possible codons was used to encode this particular serine over here or that leucine over there.

The ability to translate DNA/RNA to peptides gives us the ability to search in several ways, which translates (forgive the pun) into the following BLAST algorithms:

|  Algorithm    | Searches for<br/>(query) | Against database of<br/>(target) | Using sequence<br/>alignment of |
|:--------------|:------------------------:|:--------------------------------:|:-------------------------------:|
| **`blastn`**  | nucleotides              | nucleotides                      | nucleotides (_DNA/RNA space_) |
| **`blastp`**  | peptides                 | peptides                         | peptides (_protein space_)    |
| **`tblastn`** | peptides                 | nucleotides                      | peptides (_translated_) |
| **`blastx`**  | nucleotides              | peptides                         | peptides (_translated_) |
| **`tblastx`** | nucleotides              | nucleotides                      | peptides* (_translated x2_) |

\* yes, `tblastx` searches for nucleotides against nucleotides, but it searches in translated (i.e. peptide) space. **Question:** can you think of a reason why anybody would want to do that?

**WARNING:** There isn't just one genetic code. Mammalian _mitochondria_, for example, have their own code, which — however similar — is not the same as the mammalian _nuclear/cytosolic_ code. **Make sure you know what you're translating before you translate it!**

## What we will do
We will run BLAST to find genes that _look like_ our genes (i.e. they have significant _sequence homology_), and that therefore _may_ (or most certainly may _not_) have a similar function in our cells as they do in the cells where we found the similar one.

You'll remember we have created a `genbank` file containing both the sequences and gene positions for one of the chromosomes in the _Saccharomyces cerevisiae_ genome, and we also have FASTA files for each of those sequences.

In both of those cases we deleted all the extra information, all we know is the sequence and where in the chromosome it is.

But **what are those genes? What do they do?**

Of course we already know what they do because the genome of _S. cerevisiae_ is extremely well known, and that information was in the GFF before we decided to get rid of it. For this lesson, though, let's pretend we are working with a new organism. All we know is where the genes are, and we want to infer what those genes do starting only from their sequence.

## Before we start – Make sure you're using the right `conda` environment
If you're following along from the previous lesson, you should be here under your `cg2` environment. You can confirm this with:

If you see anything other than `cg2` being printed out above, please close this window and then hit "**Quit**" in the main Jupyter Home window (if you still have it open, no worries if not). If you don't have it open, open a new terminal window and do:<br/>
`jupyter notebook stop`<br/>

Then do the following:<br/>
`conda activate cg2`<br/>
`jupyter notebook`<br/>

If that still fails, please [send me an email](mailto:vsojo@amnh.org).

## Importing libraries & packages
We will need the following:

|Library/Package     | Purpose |
|:-------------------|:--------|
|`Bio.SeqRecord`     | To process sequences and their annotations using Biopython|
|`Bio.Seq`           | To store sequences using Biopython|
|`Bio.Seq`           | To read and write sequences using Biopython|
|`Bio.SeqFeature`    | To process genome annotations (e.g. gene positions)|
|`Bio.Blast NCBIWWW` | BLASTs against the public NCBI databases over the Internet|


If you don't have `pandas` installed, open a terminal window and do the following:<br/>
`conda install -n cg2 pandas -y`<br/>

## Bringing back our sequences and location annotations
Before we do any BLASTing, we need to get our sequences back. They are in the `genbank` file we exported in the previous lesson.

Let's re-import them using BioPython.

The record contains the long sequence of the chromosome, plus a number of specified `features`.

In the previous lesson we chose the first chromosome, which has 117 `features`. We know that each and every one of these features is a `gene`, since that's what we chose in the previous lesson. Note however that they could have been a combination of any valid features, such as ORFs, UTRs, tRNAs, and so on, as in the original GFF file.

Let's take the first `gene` (i.e. the first `feature`) from the chromosome record object:

That looks very much like the information we provided in the previous lesson, so all looking good.

We can `extract` the sequence for the fragment corresponding to this gene from the long sequence of the entire chromosome, and make a BioPython SeqRecord with all this information:

Now would be a good time to check that BioPython is doing what it's supposed to be doing. Let's extract that same information ourselves:

That looks good. Let's move on... ok ok, let's do a formal comparison:

Happy now? Let's move on.

## BLASTing the Web
You have probably done some BLASTing online on the [NCBI website](https://blast.ncbi.nlm.nih.gov/Blast.cgi).

Both the bash installation of BLAST and the BioPython version let us do something similar. Because we've already used the bash version, here we will be using BioPython (do take a look at the BioStar Handbook, the official BLAST tutorial, or the hundreds upon hudreds of unofficial tutorials online if you want to learn more about the shell version).

There are in general three steps to a BLAST search:
1. Building the target database.
2. Performing the actual BLAST of the query sequence against that database.
3. Parsing the output of the search.

In this section we will be using the public databases on the internet, so we can skip step one. We'll come back to making our own database later.

Let's BLAST the sequence of the first gene against the `nt` database at NCBI. This is the non-redundant nucleotide database (basically all publicly available DNA/RNA sequences end up in there at some point, after some curation).

Note that we gave the gene_record in `fasta` format – and not just the sequence – so that BLAST keeps the ID that we assigned above, which will make it easier for us to compare later.

**The command above will take a while**. Keep in mind that we're essentially BLASTing against the entire accumulated (high-quality and well-annotated) genetic knowledge of humankind (minus what pharma companies and lazy/unethical researchers keep for themselves).

Also, NCBI servers get more or less busy throughout the day.

### Save the result of the BLAST search to a local XML file so that you can use it later
Once the BLAST search finishes running, we should have a result as a XML text hidden inside our `result_handle`. We could work with that, but it would be much better to save it as a file in our machine so that we can reuse this information without having to perform the search again which, as you just saw, is very slow. Importantly, it also takes up resources from other people trying to use both NCBI and the Internet in general – _be a good Internet citizen_!

We no longer need the result handle from the web search, so close it:

Actually, we do need the results, but now we can load them from the XML file. Let's first take a quick look at this XML file:

From that last bit, it seems hit number one is in chromosome I of _Saccharomyces cerevisiae_. No surprises there.

Let's parse the results properly using BioPython.

## Loading and parsing the XML BLAST results
Let's load all the results for which $e_{value} < 10^{-150}$ ... $\Big(...\hat{O_h} = \hat{\Upsilon}_{es} + y^{ou} - \frac{c_an}{H(ave)} - \sum \limits _{j=1} ^{p} q_{uations}\hat{\tau}_{oo} \Big)$

We will look at all the `hsps`, or high-scoring pairs, and print a fragment of the sequences for the query (our sequence) and the subject (the sequence from the database).

It seems we're getting a bunch of matches to the entire chromosome. That's... okayish, but it would be much nicer to have genes instead.

Take a look at the `Alignment length`. The ones for complete chromosomes are extremely long. Maybe we can add a threshold to ignore any target sequences that are too long:

Well, that was only one result, but one is better than nothing, and it has 100% identity!

So, it would seem our unknown gene matches 100% some mysteryous **YAL069W** gene from a cryptic species called _Saccharomyces cerevisiae_. Wow, who would have thought!

You can go and confirm in the original GFF, and you'll see that, indeed, the first gene is the one we found. This shouldn't surprise us too much, of course.

In reality, though, we often won't be able to BLAST the sequence of our desired organism against itself, since we may be working with completely new organisms whose genomes are unknown to science. Let's take a look into something like that next.

## Running BLAST against a local custom database
We'll need to follow four steps:
1. Download the BLAST software.
1. Download a reference genome sequence against which we want to blast.
1. Build a BLASTable database using the BLAST software and the reference sequences we just downloaded.
1. Perform the actual BLAST search of our query sequences against that database.

### Import the list of gene IDs
Before we start, remember that we created a list of gene IDs in the previous lesson. Let's reuse it here.

Take a look at the first and last few gene IDs to make sure it worked:

### Download BLAST software from NCBI

Uncompress it:

Those files inside the `bin/` directory are ready to use. You can test it this way:

### Downloading reference sequences to build a local BLAST database
We will need to pick a nice reference organism or set of organisms to compare our mistery (actually, it's **_Saccharomyces cerevisiae_**) sequences to.

Mmmm... (_thinking...thinking..._) ... how about **_Schizosaccharomyces pombe_**? It is an extremely well known and studied organism whose genome is widely annotated, it is a yeast after all, and on top of everything it has a very similar name to our organism of study. So, it should do, right? I have a good feeling about this...

Let's make a directory to build our database, and download the reference sequences into it. We will get the CDSs for _Schizosaccharomyces pombe_ from **Ensembl**, at the European Bioinformatics Institute (EBI).

Let's take a look at the Fasta headers and first line of the sequences found in the first 200 lines:

This seems like a pretty good source to build a BLAST database; it has lots of very useful information about each sequence.

Let's quickly count the number of genes in there.

That's fewer than the 6600 that we found for _S. cerevisiae_. This is not a great start, and maybe we should be using several organisms as reference instead of just one, but let's move on.

### Building the local database
We first need to download the BLAST software so that we can run it locally. We will do that from [NCBI](ftp://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/). The most appropriate file for the Mac is:

There's a command called `makeblastdb` inside the `bin/` directory. Let's use it to make our database into a directory that we will create just for this purpose:

Let's take a look inside the `db/` folder

### BLAST searching against the local database of the _Schizosaccharomyces pombe_ genome

Let's create a function that will perform the BLAST search, so that we can reuse it with other reference sequences if we need to.

And now let's use the function to search our list of genes against the _S. pombe_ database we made above.

Hm... was it a good idea to choose the fission yeast _Schizosaccharomyces pombe_ as a reference for the budding yeast _Sacharomyces cerevisiae_?

It turns out these things **diverged from each other around 500 million years ago!** That's before the dinosaurs even walked the Earth... come to think about it, it's **before _anything_ walked the Earth**.

So, it would seem it was a horrendously bad idea to choose _S. pombe_ as a reference for _S. cerevisiae_ despite the similar names and the fact that they are both "yeasts".

### BLASTing against a more closely related yeast, _Ashbya gossypii_

Let's start again with a much more closely related yeast that has a very different name, _Ashbya gossypii_.

Good, we now have a second database there, let's repeat our analysis using Ashbya as a reference:

That was not a whole lot better, was it? Hm...

### Let's try against _S. cerevisiae_, then!

That looks lovely. We found a match for every single gene we tried :-)

...well, of course we did, it's the same organism!

## Why such terrible results with the other species?
It seems that, at least for this particular chromosome (chromosome 1 of the _S. cerevisiae_ genome), not a lot of hits can be found even in a relatively closely related yeast.

### So, what could we do differently?
Perhaps we could try adding many Ascomycetes (the phylum of the Fungi that all three species above belong to), or at least many Saccharomycetales (which both _Ashbya_ and _Saccharomyces_ belong to) into our BLAST database.

The question is: how many of those genomes are reliable references? There are not a whole lot of genomes in the (very reliable and carefully curated) list of [Ensemble fungal reference genomes](ftp://ftp.ensemblgenomes.org/pub/fungi/release-48/fasta/). Maybe you found another genome somewhere in the internet... but can you trust it? Often the functional annotations in genomes come precisely from BLASTing against other genomes, so **you need to be sure that there is actual experimental evidence for a gene's biological function before you use it to BLAST against**.

### Take home message
Be extremely careful with your reference genomes!

## Homework & Quiz
Please create a new Jupyter Notebook and answer the following questions:
1. Thinking evolutionarily, we were comparing DNA vs DNA of unicellular species that may have diverged over 500 million years ago, and whose generation times are relatively short. That's probably quite a lot of evolution. We could have used peptide translations instead of the gene nucleotide sequences. Why _might_ this give us a better result?
1. We do have a tool in BLAST to search nucleotides queries against a nucleotide target database, but using space, `tblastx`. Go ahead and try that with _S. pombe_ and _Ashbya_. Do the results change? How many hits do you find for both
1. Actually, the above may not be such a great idea. We have `CDS`s in our BLAST database, that's good for `tblastx`. However, for our queries, we selected `gene` in the first lesson. There are at least two things that could go very wrong with this. Can you spot at least one? Extra points if you get more than one thing that could go wrong.
1. Using the first lesson as reference, extract the CDSs instead of the genes, then run the analysis again, in peptide space (i.e. using `tblastx`).

## Finding help and more info about commands
Other than DuckDuckGoing or Googling whatever you wish, or going straight to the manuals of the tools you're using, you can find information about and object in Python by using the `help` function: